## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Punta Arenas,CL,-53.1500,-70.9167,46.51,66,75,11.50
1,1,Yellowknife,CA,62.4560,-114.3525,28.92,46,0,6.91
2,2,Kodiak,US,57.7900,-152.4072,42.31,65,40,6.91
3,3,Yulara,AU,-25.2406,130.9889,71.44,26,6,17.27
4,4,Ushuaia,AR,-54.8000,-68.3000,42.46,81,40,24.16


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Namibe,AO,-15.1961,12.1522,77.40,80,99,4.76
10,10,Mecca,SA,21.4267,39.8261,81.41,46,92,3.60
11,11,Lodwar,KE,3.1191,35.5973,83.12,59,100,18.19
13,13,Butaritari,KI,3.0707,172.7902,81.70,79,60,13.33
15,15,Kapaa,US,22.0752,-159.3190,82.38,89,100,21.85
17,17,Vaini,TO,-21.2000,-175.2000,82.56,83,75,12.66
19,19,Rikitea,PF,-23.1203,-134.9692,79.03,71,93,19.13
21,21,Carnarvon,AU,-24.8667,113.6333,78.87,29,98,9.22
27,27,Hambantota,LK,6.1241,81.1185,80.78,81,25,4.47
34,34,Kendari,ID,-3.9450,122.4989,86.50,64,100,0.89


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID       176
City          176
Country       176
Lat           176
Lng           176
Max Temp      176
Humidity      176
Cloudiness    176
Wind Speed    176
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Namibe,AO,77.40,-15.1961,12.1522,
10,Mecca,SA,81.41,21.4267,39.8261,
11,Lodwar,KE,83.12,3.1191,35.5973,
13,Butaritari,KI,81.70,3.0707,172.7902,
15,Kapaa,US,82.38,22.0752,-159.3190,
17,Vaini,TO,82.56,-21.2000,-175.2000,
19,Rikitea,PF,79.03,-23.1203,-134.9692,
21,Carnarvon,AU,78.87,-24.8667,113.6333,
27,Hambantota,LK,80.78,6.1241,81.1185,
34,Kendari,ID,86.50,-3.9450,122.4989,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.count()

City          176
Country       176
Max Temp      176
Lat           176
Lng           176
Hotel Name    176
dtype: int64

In [10]:
# 7 drop the empty rows from the hotel_df 
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)


In [11]:
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

In [12]:
hotel_df.count()

City          159
Country       159
Max Temp      159
Lat           159
Lng           159
Hotel Name    159
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file  = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))